In [1]:
import matplotlib.pyplot as plt
import PIL
import pandas as pd
import numpy as np
import calendar
import time
import requests
import datetime

pd.options.display.max_columns = 999
pd.options.display.max_rows = 999

In [62]:
df_lonely_com['created_utc']

0          1632304023
1          1632303572
2          1632303287
3          1632303182
4          1632303106
             ...     
752384    1.29593e+09
752385    1.29589e+09
752386    1.29586e+09
752387    1.29515e+09
752388    1.29428e+09
Name: created_utc, Length: 752382, dtype: object

# Filter data

In [3]:
# read in raw data
df_loneliness_sub = pd.read_csv("data/loneliness_submission_0921.csv")
df_lonely_sub = pd.read_csv("data/lonely_submission_0921.csv")
df_fa_sub = pd.read_csv("data/fa_submission_0921.csv")
df_isolation_sub = pd.read_csv("data/isolation_submission_0921.csv")

df_loneliness_com = pd.read_csv("data/loneliness_comments_0921.csv")
df_lonely_com = pd.read_csv("data/lonely_comments_0921.csv")
df_fa_com = pd.read_csv("data/fa_comments_0921.csv")
df_isolation_com = pd.read_csv("data/isolation_comments_0921.csv")

/Users/carolcheng/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (0,1,4,5,6,8,9,10,11,12,13,17,19,20,21,22,23,24,26,27,28,29,30,31,32,33,34,35,39,41,46,47,52,56,59,61,62,63,64,65,66,67,70,71,72,73,76,80,81,83,86,89,90,93,94,96,97,98) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/carolcheng/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (0,1,4,6,7,8,9,10,11,12,13,17,19,20,21,22,23,24,25,26,28,29,30,31,32,33,36,37,39,44,45,46,50,54,57,59,60,61,62,63,64,65,66,69,74,75,76,78,79,81,83,84,85,86,87,89,90,91,92,94,97,98,99,100,102,103,104) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/carolcheng/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: 

In [7]:
print(*df_lonely_sub.columns, sep = "\t")

all_awardings	allow_live_comments	author	author_flair_css_class	author_flair_richtext	author_flair_text	author_flair_type	author_fullname	author_is_blocked	author_patreon_flair	author_premium	awarders	can_mod_post	contest_mode	created_utc	domain	full_link	gildings	id	is_created_from_ads_ui	is_crosspostable	is_meta	is_original_content	is_reddit_media_domain	is_robot_indexable	is_self	is_video	link_flair_background_color	link_flair_richtext	link_flair_template_id	link_flair_text	link_flair_text_color	link_flair_type	locked	media_only	no_follow	num_comments	num_crossposts	over_18	parent_whitelist_status	permalink	pinned	pwls	retrieved_on	score	selftext	send_replies	spoiler	stickied	subreddit	subreddit_id	subreddit_subscribers	subreddit_type	thumbnail	title	total_awards_received	treatment_tags	upvote_ratio	url	whitelist_status	wls	post_hint	preview	removed_by_category	author_flair_background_color	author_flair_text_color	banned_by	author_cakeday	edited	gilded	distinguished	steward_reports	

## Submission

In [53]:
name_lst = ["lonely", "loneliness", "ForeverAlone", "isolation"]
sub_lst = [df_lonely_sub, df_loneliness_sub, df_fa_sub, df_isolation_sub]

for i, df in enumerate(sub_lst):
    file_name = "clean-data/{}_sub_clean_0921.csv".format(name_lst[i])
    print("creating ", file_name)
    df_sub_clean_with_empty = df[['subreddit',
                                  'created_utc',
                                  'id',
                                  'title',
                                  'author',
                                  'author_fullname',
                                  'num_comments',
                                  'upvote_ratio',
                                  'num_crossposts',
                                  #'link_flair_text',
                                  'over_18',
                                  'total_awards_received',
                                  'selftext',
                                  'domain',
                                  'full_link',
                                  'permalink']]
    print(df_sub_clean_with_empty.shape)
    df_sub_clean_with_empty['selftext'].replace('',np.nan,inplace=True)
    df_sub_clean_with_empty.dropna(subset=['selftext'],inplace=True) # drop empty content
    print(df_sub_clean_with_empty.shape)
    df_sub_clean_with_empty.to_csv(file_name,index=False)

creating  clean-data/lonely_sub_clean_0921.csv
(115365, 15)
(110937, 15)


/Users/carolcheng/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


creating  clean-data/loneliness_sub_clean_0921.csv
(4213, 15)
(3964, 15)
creating  clean-data/ForeverAlone_sub_clean_0921.csv
(118118, 15)
(97254, 15)
creating  clean-data/isolation_sub_clean_0921.csv
(141, 15)
(74, 15)


## comments

In [60]:
name_lst = ["lonely", "loneliness", "ForeverAlone", "isolation"]
com_lst = [df_lonely_com, df_loneliness_com, df_fa_com, df_isolation_com]

for i, df in enumerate(com_lst):
    file_name = "clean-data/{}_com_clean_0921.csv".format(name_lst[i])
    print("creating ", file_name)
    
    df_com_clean_with_empty = df[['subreddit',
         'created_utc',
         'id',
         'link_id',
         'parent_id',
         'author',
         'author_fullname',
         'body',
         'collapsed',
         'is_submitter',
         'no_follow',
         'permalink'
        ]]
    print(df_com_clean_with_empty.shape)
    df_com_clean_with_empty['body'].replace('',np.nan,inplace=True)
    df_com_clean_with_empty.dropna(subset=['body','created_utc'],inplace=True) # drop empty content
    print(df_com_clean_with_empty.shape)
    df_com_clean_with_empty.to_csv(file_name,index=False)

creating  clean-data/lonely_com_clean_0921.csv
(752382, 12)


/Users/carolcheng/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


(738710, 12)
creating  clean-data/loneliness_com_clean_0921.csv
(21909, 12)
(21909, 12)
creating  clean-data/ForeverAlone_com_clean_0921.csv
(2074640, 12)
(2054757, 12)
creating  clean-data/isolation_com_clean_0921.csv
(187, 12)
(187, 12)


In [50]:
df_fa_sub['link_flair_text'].value_counts()

Vent                   11603
Advice Wanted           3346
Success Story           1205
Trigger Warning            9
Trigger warning            1
Successful Vent            1
SPOILERS                   1
not appropriate sub        1
no content                 1
Under 30? fuck off         1
META                       1
Incel Approved             1
wrong subreddit            1
Name: link_flair_text, dtype: int64

In [48]:
df_sub_clean_with_empty.head()

,subreddit,created_utc,id,title,author,author_fullname,num_comments,upvote_ratio,num_crossposts,over_18,total_awards_received,selftext,domain,full_link,permalink
1,loneliness,1632241037,psm2m7,My battle with depression and trying to stay a...,brokenandtired1,t2_8esl7btb,0,1.0,0.0,False,0.0,I don't know if this is allowed but I struggle...,self.loneliness,https://www.reddit.com/r/loneliness/comments/p...,/r/loneliness/comments/psm2m7/my_battle_with_d...
2,loneliness,1632104242,prln82,My Friends Don't Get Me,Felicino,t2_cjcbjdfs,0,1.0,0.0,False,0.0,Sometimes I think about getting rid of all of ...,self.loneliness,https://www.reddit.com/r/loneliness/comments/p...,/r/loneliness/comments/prln82/my_friends_dont_...
3,loneliness,1632094625,prj4ob,NO COST TELEHEALTH INTERVENTION FOR LONELINESS,utmindfulnessstudy,t2_6lrnthdr,0,1.0,0.0,False,0.0,Have you been struggling with feelings of lone...,self.loneliness,https://www.reddit.com/r/loneliness/comments/p...,/r/loneliness/comments/prj4ob/no_cost_teleheal...
4,loneliness,1632076962,prdryx,I am fucking tired of masturbation,Carlador,t2_66fi2rmu,10,1.0,0.0,False,0.0,"You hop on your boat, find heaven and then eve...",self.loneliness,https://www.reddit.com/r/loneliness/comments/p...,/r/loneliness/comments/prdryx/i_am_fucking_tir...
5,loneliness,1632075670,prdczh,Curious about how your soulmate looks like? Tr...,Gusterul23,t2_16nfc8,4,1.0,0.0,False,0.0,Ever wondered how your soulmate looks like? Be...,self.loneliness,https://www.reddit.com/r/loneliness/comments/p...,/r/loneliness/comments/prdczh/curious_about_ho...
